<a href="https://colab.research.google.com/github/markusMiksa1304/pubColab/blob/main/KWbq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Intro

**Colab Authentication**

Run this first if you want to use the interactive Queries (not needed for read only!)

In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
# shapley model code for optional #python_feature-analysis 
!pip install shap

Test der Verbindung

In [ ]:
%%bigquery --project tefgfs
#https://towardsdatascience.com/leveraging-bigquery-with-google-analytics-data-b6085d8415b


WITH
  dateRangeData AS (
  SELECT    *  FROM   `tefgfs.103498486.ga_sessions_20201005`)   #  WHERE  _TABLE_SUFFIX BETWEEN "20201005" AND "20201007"  )


Select
Count ( Distinct(
CASE
 WHEN totals.visits=1 THEN   #warum 
      CONCAT( fullvisitorid,"-",CAST(visitStartTime AS string))    
else ""    
End )) as Sessions
from
dateRangeData#, UNNEST(hits) AS hits#,


---
# **InputDaten mopGA6**  .. plus Order_Status


ein Blick auf die beiden Datasets mmML und ReportingData

In [ ]:
%%bigquery --project tefgfs 
SELECT
 * EXCEPT(is_typed)
FROM
 tefgfs.mmML.INFORMATION_SCHEMA.TABLES
# tefgfs.ReportingData.INFORMATION_SCHEMA.TABLES
#WHERE
# table_type="BASE TABLE"

In [ ]:
%%bigquery --project tefgfs 
WITH
  mopGA AS ( SELECT  if(gaTransactId is null,0,1) as label, *  FROM   `tefgfs.ReportingData.mopGaSourcev6_o2_20200829` limit 10 )      
Select * from mopGA

In [ ]:
%%bigquery --project tefgfs 
WITH
  mopGA AS (
  SELECT    *  FROM   `tefgfs.ReportingData.mopGaSourcev6_o2_20200829` where gaTransactId <> "None")   #  WHERE  _TABLE_SUFFIX BETWEEN "20201005" AND "20201007"  )

Select gaTransactId,TRANSACTION_ID,ORDER_STATUS 
from (
 SELECT * FROM  mopGA #where gaTransactId is not NULL  and gaTransactId <> "None" Limit 10) mopGA
LEFT  JOIN  (SELECT * FROM `tefgfs.MetaLookupData.NET_Conversions_*`   )  Activations
ON mopGA.gaTransactId = Activations.TRANSACTION_ID ) mopGA6plus  



---
# **MachineLearning aufsetzen**


**Hinzufügen des Orderaktivierungs-Status zur mopGA**

In [ ]:
%%bigquery --project tefgfs 

SELECT *   EXCEPT (TRANSACTION_ID,UPLOAD_DATE	) FROM  (Select *   From `tefgfs.ReportingData.mopGaSourcev6_o2_20200829`  where gaTransactId is not NULL  and gaTransactId <> "None" ) mopGA
LEFT  JOIN  (SELECT * FROM `tefgfs.MetaLookupData.NET_Conversions_*`   )  Activations
ON mopGA.gaTransactId = Activations.TRANSACTION_ID   Where ORDER_STATUS <> "None" order by ORDER_STATUS

**Hinzufügen weiterer Spalten zur mopGA**

In [ ]:
%%bigquery --project tefgfs 

SELECT *   EXCEPT (TRANSACTION_ID,UPLOAD_DATE	) FROM  (Select *   From `tefgfs.ReportingData.mopGaSourcev6_o2_20200829`  where gaTransactId is not NULL  and gaTransactId <> "None" ) mopGA
LEFT  JOIN  (SELECT * FROM `tefgfs.MetaLookupData.NET_Conversions_*`   )  Activations
ON mopGA.gaTransactId = Activations.TRANSACTION_ID   Where ORDER_STATUS <> "None" order by ORDER_STATUS

**Rauschen entfernen und  bouncer**

In [ ]:
%%bigquery --project tefgfs 

Select  count(*) from (Select if(gaTransactId is null,0,1) as label, *  FROM   `tefgfs.ReportingData.mopGaSourcev6_o2_20200829` ) 
where   sessionTotalInteractionCount > 4 and sessionDurationMS > 500  and isBounce	= 0   # and  label = 1    


**Trainingsdaten**-Tabelle 

In [ ]:
%%bigquery --project tefgfs 
-- Create train  data 10 Days

CREATE  OR REPLACE Table `tefgfs.mmML.KW_trainData` as

Select *  EXCEPT(sessionDate, rowId,visitId,GCLID, gaTransactId,transactionItemCats,transactionItemVariants,isCartAdd, isDirectBuySession,isEShopTransaction,isSessionCommProduct,	isSessionCd70Postpaid,     quantity,ProductCategoryFilter_PO             
                , productCheckoutPromocode) 
from (Select if(gaTransactId is null,0,1) as label, *  FROM   `tefgfs.ReportingData.mopGaSourcev6_o2_*`  WHERE  _TABLE_SUFFIX BETWEEN "20200920" AND "20200930" ) 
where   sessionTotalInteractionCount > 4 and sessionDurationMS > 500  and isBounce	= 0



In [ ]:
%%bigquery --project tefgfs 
Select count(label) as yLines from `tefgfs.mmML.KW_trainData` 

Sichtung der Trainingsdaten

In [ ]:
%%bigquery --project tefgfs 
Select * from `tefgfs.mmML.KW_trainData` limit 10

**Test Daten**-Tabelle

In [ ]:
%%bigquery --project tefgfs 
-- Create train  data .

CREATE  OR REPLACE Table `tefgfs.mmML.KW_testData` as

Select *  EXCEPT(sessionDate, rowId,visitId,GCLID, gaTransactId,transactionItemCats,transactionItemVariants,isCartAdd, isDirectBuySession,isEShopTransaction,isSessionCommProduct,	isSessionCd70Postpaid, quantity,ProductCategoryFilter_PO
           ,productCheckoutPromocode) 
from (Select if(gaTransactId is null,0,1) as label, *  FROM   `tefgfs.ReportingData.mopGaSourcev6_o2_*`  WHERE  _TABLE_SUFFIX BETWEEN "20201001" AND "20201007" ) 
where   sessionTotalInteractionCount > 4 and sessionDurationMS > 500  and isBounce	= 0

In [ ]:
%%bigquery --project tefgfs 
Select count(label) as yLines from `tefgfs.mmML.KW_testData` 

In [ ]:
%%bigquery --project tefgfs 

-- Create veryLarge TestData2

CREATE  OR REPLACE Table `tefgfs.mmML.KW_testData2` as

Select *  EXCEPT(sessionDate, rowId,visitId,GCLID, gaTransactId,transactionItemCats,transactionItemVariants,isCartAdd, isDirectBuySession,isEShopTransaction,isSessionCommProduct,	isSessionCd70Postpaid, quantity,ProductCategoryFilter_PO
, productCheckoutPromocode) 
from (Select if(gaTransactId is null,0,1) as label, *  FROM   `tefgfs.ReportingData.mopGaSourcev6_o2_*`  WHERE  _TABLE_SUFFIX BETWEEN "20200501" AND "20200801" ) 
where   sessionTotalInteractionCount > 4 and sessionDurationMS > 500  and isBounce	= 0


In [ ]:
%%bigquery --project tefgfs 
Select count(label) as yLines from `tefgfs.mmML.KW_testData2` 


# **Klassisches python** mit shap 
https://slundberg.github.io/shap/notebooks/NHANES%20I%20Survival%20Model.html

In [ ]:
import shap
import xgboost
from sklearn.model_selection import train_test_split
import matplotlib.pylab as pl

In [ ]:
#hot encoding
#https://www.kaggle.com/ankitkuls/xgboost-with-one-hot-encoding
#https://towardsdatascience.com/explaining-feature-importance-by-example-of-a-random-forest-d9166011959e
#https://medium.com/applied-data-science/new-r-package-the-xgboost-explainer-51dd7d1aa211
#python_feature-analysis:
#https://machinelearningmastery.com/feature-importance-and-feature-selection-with-xgboost-in-python/


...


# **BQ xgboost**


In [ ]:
%%bigquery --project tefgfs 

CREATE  OR REPLACE MODEL `tefgfs.mmML.KW_xgboostMopGAv6`
OPTIONS(MODEL_TYPE='BOOSTED_TREE_CLASSIFIER',
        BOOSTER_TYPE = 'GBTREE',
        NUM_PARALLEL_TREE = 1,
        MAX_ITERATIONS = 50,
        TREE_METHOD = 'HIST',
        EARLY_STOP = TRUE,
        SUBSAMPLE = 0.85,
        INPUT_LABEL_COLS = ['label']) AS 
SELECT * from `tefgfs.mmML.KW_trainData`  

**xgb Auswertung**

In [ ]:
%%bigquery  --project tefgfs 

SELECT
  *
FROM
  ML.TRAINING_INFO(MODEL `tefgfs.mmML.KW_xgboostMopGAv6`)
ORDER BY iteration 


In [ ]:
%%bigquery  --project tefgfs 
SELECT
  *
FROM
  ML.FEATURE_INFO(MODEL `tefgfs.mmML.KW_xgboostMopGAv6`)

Model **evaluate**

In [ ]:
%%bigquery --project tefgfs 

SELECT
  *
FROM ML.EVALUATE(MODEL `tefgfs.mmML.KW_xgboostMopGAv6`,  
(Select * from  `tefgfs.mmML.KW_testData2`) )


In [ ]:
%%bigquery --project tefgfs 

SELECT
  *,
  CASE
    WHEN roc_auc > .9 THEN 'good'
    WHEN roc_auc > .8 THEN 'fair'
    WHEN roc_auc > .7 THEN 'decent'
    WHEN roc_auc > .6 THEN 'not great'
  ELSE 'poor' END AS model_quality
FROM 
ML.EVALUATE(MODEL `tefgfs.mmML.KW_xgboostMopGAv6`,  
(Select * from  `tefgfs.mmML.KW_testData2`) )

#0.995145

**Die ROC Kurve**

In [ ]:
%%bigquery --project tefgfs roc
SELECT
  * 
FROM
  ML.ROC_CURVE(MODEL `tefgfs.mmML.KW_xgboostMopGAv6`)

In [ ]:
# and plot our ROC curve!
import matplotlib.pyplot as plt

# plot the false positive rate by true postive rate (aka recall)
plt.plot(roc.false_positive_rate, roc.recall)

**Feature importance**

In [ ]:
%%bigquery  --project tefgfs Fimportance

SELECT
  *
FROM
  ML.FEATURE_IMPORTANCE(MODEL `tefgfs.mmML.KW_xgboostMopGAv6`) order by importance_weight desc



In [ ]:
# horizontaler barchart zur Fimportance der 10 feature
import matplotlib.pyplot as plt
import numpy as np

Fimportance
cols = ['feature','importance_weight']
fimportance = Fimportance[cols].head(10)
featureNames = fimportance['feature']
importance = fimportance['importance_weight']

plt.rcdefaults()
fig, ax = plt.subplots()

y_pos = np.arange(len(featureNames))

#performance = 3 + 10 * np.random.rand(len(people))
#error = np.random.rand(len(people))

#ax.barh(y_pos, performance, xerr=error, align='center')
ax.barh(y_pos, importance,  align='center')

ax.set_yticks(y_pos)
ax.set_yticklabels(featureNames)
ax.invert_yaxis()  # labels read top-to-bottom
ax.set_xlabel('importance')
ax.set_title('FeatureImportance')

plt.show()

In [ ]:
Fimportance


die **für das Modell relevantesten Feature** (also die Spalten der TraingsdatenTabelle) sind:


In [ ]:
featureNames

**die für die Testdaten prognostizierte Kaufwahrscheinlichkeit**


In [ ]:
#schreibe die Ergebnistabelle

%%bigquery --project tefgfs 

CREATE  OR REPLACE Table `tefgfs.mmML.KW_resultData` as
SELECT
  *
FROM ML.PREDICT(MODEL `tefgfs.mmML.KW_xgboostMopGAv6`,  
(Select * from  `tefgfs.mmML.KW_testData`) )

In [ ]:
#zeige die KW der Ergebnistabelle an 
%%bigquery --project tefgfs 

Select  trueLabel,predicted_label, 1-predicted_label_probs[OFFSET(1)].prob as KW,
from
(SELECT  o.label as trueLabel,*   FROM `tefgfs.mmML.KW_resultData` as o,Unnest(predicted_label_probs ) as p limit 100)  order by trueLabel desc

# **AutoML useCases**


In [ ]:
%%bigquery --project tefgfs 

CREATE OR REPLACE MODEL `tefgfs.mmML.autoMopGAv6`
OPTIONS(MODEL_TYPE = 'automl_classifier', budget_hours=2.0
       ) AS 
Select * from (Select if(gaTransactId is null,0,1) as label, *  FROM   `tefgfs.ReportingData.mopGaSourcev6_o2_20200828` ) 
where   sessionTotalInteractionCount > 4 and sessionDurationMS > 500  


 

**Auswertung**

In [ ]:
%%bigquery --project tefgfs 

SELECT
  *
FROM
  ML.TRAINING_INFO(MODEL`tefgfs.mmML.logMopGAv6`)
ORDER BY iteration 


Model **evaluate**

In [ ]:
%%bigquery  --project tefgfs 

SELECT
  *
FROM ML.EVALUATE(MODEL `tefgfs.mmML.logMopGAv6`, 
Select * from (Select if(gaTransactId is null,0,1) as label, *  FROM   `tefgfs.ReportingData.mopGaSourcev6_o2_20200829` ) 
where   sessionTotalInteractionCount > 4 and sessionDurationMS > 500  )

# **linReg useCases**


erst mal ein einfaches linReg-Model zur Vorhersage der total.transactions
https://medium.com/@davide.anastasia/bigquery-ml-and-a-kaggle-competition-1f288cc2df65

Numerical variables (of type NUMERIC, FLOAT64 or INT64) are standardized by default. In addition, standardization is automatically applied during prediction
Features
In order to “force” BigQuery to use **one-hot enconding**, even if the columns are numeric, we can use CAST to STRING. This simple change brought the evaluation score from 0.0137 to 0.0103 with half the number of iterations.
CAST(app AS STRING) as app

CREATE MODEL `kaggle_talkingdata_adtracking.talkingdata_logreg_sample_0003`
OPTIONS (
  model_type='logistic_reg',
  input_label_cols=['is_attributed'],
  data_split_method='seq',
  data_split_col='click_time'
) AS
SELECT CAST(ip AS STRING) as ip,
  CAST(app AS STRING) as app,
  CAST(device AS STRING) as device,
  CAST(os AS STRING) as os,
  CAST(channel AS STRING) as channel,
  click_time,
  is_attributed
FROM `kaggle_talkingdata_adtracking.dataset_train_sample`;

In [ ]:
 %%bigquery --project tefgfs 
CREATE MODEL IF NOT EXISTS `tefgfs.mmML.linRegGA`
OPTIONS(model_type='logistic_reg') AS
SELECT
  IF(totals.transactions IS NULL, 0, 1) AS label,
  IFNULL(device.operatingSystem, "") AS os,
  device.isMobile AS is_mobile,
  IFNULL(geoNetwork.country, "") AS country,
  IFNULL(totals.pageviews, 0) AS pageviews
FROM
     `tefgfs.103498486.ga_sessions_*`   
WHERE
  _TABLE_SUFFIX BETWEEN "20201005" AND "20201007"  


In [ ]:
#Model2:

The following query creates a model and specifies model options. Run this query to train your model:
CREATE OR REPLACE MODEL `ecommerce.classification_model`
%%bigquery  --project tefgfs 

CREATE MODEL IF NOT EXISTS `tefgfs.mmML.linRegGA2`
OPTIONS
(
model_type='logistic_reg',
labels = ['will_buy_on_return_visit']
)
AS
with dataRange as
(
  
  SELECT    *  FROM   `tefgfs.103498486.ga_sessions_20201005`#  WHERE  _TABLE_SUFFIX BETWEEN "20200920" AND "20200930"  
)
#standardSQL
SELECT
  * EXCEPT(fullVisitorId)
FROM

  # features
  (SELECT
    fullVisitorId,
    IFNULL(totals.bounces, 0) AS bounces,
    IFNULL(totals.timeOnSite, 0) AS time_on_site
  FROM
    dataRange
  WHERE
    totals.newVisits = 1
  )
  JOIN
  (SELECT
    fullvisitorid,
    IF(COUNTIF(totals.transactions > 0 AND totals.newVisits IS NULL) > 0, 1, 0) AS will_buy_on_return_visit
  FROM
     dataRange
  GROUP BY fullvisitorid)
  USING (fullVisitorId)




**Test** 

In [ ]:
#Test Model2

%%bigquery --project tefgfs 

SELECT
  roc_auc,
  CASE
    WHEN roc_auc > .9 THEN 'good'
    WHEN roc_auc > .8 THEN 'fair'
    WHEN roc_auc > .7 THEN 'decent'
    WHEN roc_auc > .6 THEN 'not great'
  ELSE 'poor' END AS model_quality
FROM
  ML.EVALUATE(MODEL `tefgfs.mmML.linRegGA2`,  (

with testdataRange as
(
  
  SELECT    *  FROM   `tefgfs.103498486.ga_sessions_20201006`#  WHERE  _TABLE_SUFFIX BETWEEN "20200920" AND "20200930"  
)
SELECT
  * EXCEPT(fullVisitorId)
FROM

  # features
  (SELECT
    fullVisitorId,
    IFNULL(totals.bounces, 0) AS bounces,
    IFNULL(totals.timeOnSite, 0) AS time_on_site
  FROM
   testdataRange
  WHERE
    totals.newVisits = 1
    )
  JOIN
  (SELECT
    fullvisitorid,
    IF(COUNTIF(totals.transactions > 0 AND totals.newVisits IS NULL) > 0, 1, 0) AS will_buy_on_return_visit
  FROM
      testdataRange
  GROUP BY fullvisitorid)
  USING (fullVisitorId)

))


**Model3**

---


A key new feature that was added to the training dataset query is the maximum checkout progress each visitor reached in their session, which is recorded in the field **hits.eCommerceAction.action_type**. If you search for that field in the field definitions you will see the field mapping of 6 = Completed Purchase.
The web analytics dataset has nested and repeated fields like ARRAYS which need to broken apart into separate rows in your dataset. This is accomplished by using the UNNEST() function, which you can see in the above query.


In [ ]:
#Train Model3

%%bigquery --project tefgfs 
#Train Model3


CREATE OR REPLACE MODEL `tefgfs.mmML.linRegGA3`
OPTIONS
  (model_type='logistic_reg', labels = ['will_buy_on_return_visit']) AS

WITH dataRange as
(
  SELECT    *  FROM   `tefgfs.103498486.ga_sessions_*` WHERE  _TABLE_SUFFIX BETWEEN "20200920" AND "20200930"  
)
,all_visitor_stats AS (
SELECT
  fullvisitorid,
  IF(COUNTIF(totals.transactions > 0 AND totals.newVisits IS NULL) > 0, 1, 0) AS will_buy_on_return_visit
  FROM dataRange
  GROUP BY fullvisitorid
)
# add in new features
SELECT * EXCEPT(unique_session_id) FROM (

  SELECT
      CONCAT(fullvisitorid, CAST(visitId AS STRING)) AS unique_session_id,

      # labels
      will_buy_on_return_visit,

      MAX(CAST(h.eCommerceAction.action_type AS INT64)) AS latest_ecommerce_progress,

      # behavior on the site
      IFNULL(totals.bounces, 0) AS bounces,
      IFNULL(totals.timeOnSite, 0) AS time_on_site,
      totals.pageviews,

      # where the visitor came from
      trafficSource.source,
      trafficSource.medium,
      channelGrouping,

      # mobile or desktop
      device.deviceCategory,

      # geographic
      IFNULL(geoNetwork.country, "") AS country

  FROM dataRange,
     UNNEST(hits) AS h

    JOIN all_visitor_stats USING(fullvisitorid)

  WHERE 1=1
    # only predict for new visits
    AND totals.newVisits = 1
    
  GROUP BY
  unique_session_id,
  will_buy_on_return_visit,
  bounces,
  time_on_site,
  totals.pageviews,
  trafficSource.source,
  trafficSource.medium,
  channelGrouping,
  device.deviceCategory,
  country
)


**Test**

In [ ]:

#Test Model3
%%bigquery --project tefgfs 
#standardSQL

SELECT 
  *,
  CASE
    WHEN roc_auc > .9 THEN 'good'
    WHEN roc_auc > .8 THEN 'fair'
    WHEN roc_auc > .7 THEN 'decent'
    WHEN roc_auc > .6 THEN 'not great'
  ELSE 'poor' END AS model_quality
FROM
  ML.EVALUATE(MODEL `tefgfs.mmML.linRegGA3`,(  

WITH  dataRange as
(
  SELECT    *  FROM   `tefgfs.103498486.ga_sessions_*` WHERE  _TABLE_SUFFIX BETWEEN "20201001" AND "20201007" 
),
all_visitor_stats AS (
SELECT
  fullvisitorid,
  IF(COUNTIF(totals.transactions > 0 AND totals.newVisits IS NULL) > 0, 1, 0) AS will_buy_on_return_visit
  FROM dataRange
  GROUP BY fullvisitorid
)

# add in new features
SELECT * EXCEPT(unique_session_id) FROM (

  SELECT
      CONCAT(fullvisitorid, CAST(visitId AS STRING)) AS unique_session_id,

      # labels
      will_buy_on_return_visit,

      MAX(CAST(h.eCommerceAction.action_type AS INT64)) AS latest_ecommerce_progress,

      # behavior on the site
      IFNULL(totals.bounces, 0) AS bounces,
      IFNULL(totals.timeOnSite, 0) AS time_on_site,
      totals.pageviews,

      # where the visitor came from
      trafficSource.source,
      trafficSource.medium,
      channelGrouping,

      # mobile or desktop
      device.deviceCategory,

      # geographic
      IFNULL(geoNetwork.country, "") AS country

  FROM dataRange,
     UNNEST(hits) AS h

    JOIN all_visitor_stats USING(fullvisitorid)

  WHERE 1=1
    # only predict for new visits
    AND totals.newVisits = 1
    
  GROUP BY
  unique_session_id,
  will_buy_on_return_visit,
  bounces,
  time_on_site,
  totals.pageviews,
  trafficSource.source,
  trafficSource.medium,
  channelGrouping,
  device.deviceCategory,
  country
)
))


roc_auch:  0.725  .. wird auch nicht mehr durch 2 weitere Parameter verbessert - auch nicht durch Ausdehung des Trainingszeitraums. 

**Austausch des Algorithmus .. von logistic_reg auf xgboost**

---




In [ ]:
%%bigquery --project tefgfs 


#Train Model3


CREATE OR REPLACE MODEL `tefgfs.mmML.linRegGA4_xgb`
#OPTIONS
#  (model_type='logistic_reg', labels = ['will_buy_on_return_visit']) AS
OPTIONS(MODEL_TYPE='BOOSTED_TREE_CLASSIFIER',
        BOOSTER_TYPE = 'GBTREE',
        NUM_PARALLEL_TREE = 1,
        MAX_ITERATIONS = 50,
        TREE_METHOD = 'HIST',
        EARLY_STOP = TRUE,
        SUBSAMPLE = 0.85,
        INPUT_LABEL_COLS = ['will_buy_on_return_visit']) AS 
WITH dataRange as
(
  SELECT    *  FROM   `tefgfs.103498486.ga_sessions_*` WHERE  _TABLE_SUFFIX BETWEEN "20200901" AND "20200930"  
)
,all_visitor_stats AS (
SELECT
  fullvisitorid,
  IF(COUNTIF(totals.transactions > 0 AND totals.newVisits IS NULL) > 0, 1, 0) AS will_buy_on_return_visit
  FROM dataRange
  GROUP BY fullvisitorid
)
# add in new features
SELECT * EXCEPT(unique_session_id) FROM (

  SELECT
      CONCAT(fullvisitorid, CAST(visitId AS STRING)) AS unique_session_id,

      # labels
      will_buy_on_return_visit,
  IFNULL(device.operatingSystem, "") AS os,
  device.isMobile AS is_mobile,
  
      MAX(CAST(h.eCommerceAction.action_type AS INT64)) AS latest_ecommerce_progress,

      # behavior on the site
      IFNULL(totals.bounces, 0) AS bounces,
      IFNULL(totals.timeOnSite, 0) AS time_on_site,
      totals.pageviews,

      # where the visitor came from
      trafficSource.source,
      trafficSource.medium,
      channelGrouping,

      # mobile or desktop
      device.deviceCategory,

      # geographic
      IFNULL(geoNetwork.country, "") AS country

  FROM dataRange,
     UNNEST(hits) AS h

    JOIN all_visitor_stats USING(fullvisitorid)

  WHERE 1=1
    # only predict for new visits
    AND totals.newVisits = 1
    
  GROUP BY
  unique_session_id,
  will_buy_on_return_visit,
  bounces,
  time_on_site,
  totals.pageviews,
  trafficSource.source,
  trafficSource.medium,
  channelGrouping,
  device.deviceCategory,
  country,
  os,is_mobile
)

 

**Auswertung**

In [ ]:
%%bigquery  --project tefgfs 

SELECT
  *
FROM
  ML.TRAINING_INFO(MODEL`tefgfs.mmML.KW_logMopGAv6`)
ORDER BY iteration 


***Test***

In [ ]:
#Test Model3
%%bigquery  --project tefgfs 
#standardSQL

SELECT 
  *,
  CASE
    WHEN roc_auc > .9 THEN 'good'
    WHEN roc_auc > .8 THEN 'fair'
    WHEN roc_auc > .7 THEN 'decent'
    WHEN roc_auc > .6 THEN 'not great'
  ELSE 'poor' END AS model_quality
FROM
  ML.EVALUATE(MODEL `tefgfs.mmML.linRegGA4_xgb`,(  

WITH  dataRange as
(
  SELECT    *  FROM   `tefgfs.103498486.ga_sessions_*` WHERE  _TABLE_SUFFIX BETWEEN "20201001" AND "20201007" 
),
all_visitor_stats AS (
SELECT
  fullvisitorid,
  IF(COUNTIF(totals.transactions > 0 AND totals.newVisits IS NULL) > 0, 1, 0) AS will_buy_on_return_visit
  FROM dataRange
  GROUP BY fullvisitorid
)

# add in new features
SELECT * EXCEPT(unique_session_id) FROM (

  SELECT
      CONCAT(fullvisitorid, CAST(visitId AS STRING)) AS unique_session_id,

      # labels
      will_buy_on_return_visit,
 IFNULL(device.operatingSystem, "") AS os,
  device.isMobile AS is_mobile,
      MAX(CAST(h.eCommerceAction.action_type AS INT64)) AS latest_ecommerce_progress,

      # behavior on the site
      IFNULL(totals.bounces, 0) AS bounces,
      IFNULL(totals.timeOnSite, 0) AS time_on_site,
      totals.pageviews,

      # where the visitor came from
      trafficSource.source,
      trafficSource.medium,
      channelGrouping,

      # mobile or desktop
      device.deviceCategory,

      # geographic
      IFNULL(geoNetwork.country, "") AS country

  FROM dataRange,
     UNNEST(hits) AS h

    JOIN all_visitor_stats USING(fullvisitorid)

  WHERE 1=1
    # only predict for new visits
    AND totals.newVisits = 1
    
  GROUP BY
  unique_session_id,
  will_buy_on_return_visit,
  bounces,
  time_on_site,
  totals.pageviews,
  trafficSource.source,
  trafficSource.medium,
  channelGrouping,
  device.deviceCategory,
  country,
  os,is_mobile
)
))

# 0.7277  statt 0.725 mit lin Reg ... lohnt fast nicht !!

**FeatureImportance**

In [ ]:
%%bigquery  --project tefgfs 

SELECT
  *
FROM
  ML.FEATURE_IMPORTANCE(MODEL `tefgfs.mmML.linRegGA4_xgb`) order by importance_weight desc #`tefgfs.mmML.linRegGA4_xgb`
